# 1 Introducción
El siguiente cuaderno realiza la suma de dos vectores, utilizando GPGPU. El algoritmo está basado en la función axpy nivel 1[3], de la biblioteca BLAS[4] que resuelve la ecuación:
<center>$Y=\alpha X + Y$</center>

Su objetivo es enseñar a los alumnos como se utiliza Python [2] la plataforma Colab[1] y CUDA[5,6]. Mostrando el funcionamiento y granularidad (grilla, bloque, warps) de sobre una dimensión (x).

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo
Ejecuta el Código CPU - GPU.

In [ ]:
# ------------------------------------------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
cantObjetos = 10000 #@param {type:"slider", min:1000, max:10000, step:1}
velocidadMaxima =   200#@param {type: "integer", min:0}
# ------------------------------------------------------------------------------

from datetime import datetime
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy

tiempo_total = datetime.now()

# ------------------------------------------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos --------------
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# ------------------------------------------------------------------------------

# Cota para la masa ------------------------------------------------------------
masaMax = 100

try: 
  if velocidadMaxima <= 0:
    raise ValueError("La velocidad maxima debe ser mayor a 0.");

  masa_cpu = numpy.random.randint(1, masaMax, size=cantObjetos)
  masa_cpu = masa_cpu.astype(numpy.int32())

  velocidad_cpu = numpy.random.uniform(1, velocidadMaxima, size=cantObjetos)
  velocidad_gpu = velocidad_cpu.astype(numpy.float32())

  energia_cpu = numpy.empty_like(velocidad_cpu)

  # CPU - reservo la memoria GPU -----------------------------------------------
  masa_gpu = cuda.mem_alloc(masa_cpu.nbytes)
  velocidad_gpu = cuda.mem_alloc(velocidad_cpu.nbytes)
  energia_gpu = cuda.mem_alloc(energia_cpu.nbytes)

  # GPU - Copio la memoria al GPU ----------------------------------------------
  cuda.memcpy_htod(masa_gpu, masa_cpu)
  cuda.memcpy_htod(velocidad_gpu, velocidad_cpu)
  cuda.memcpy_htod(energia_gpu, energia_cpu)

  # CPU - Defino la función kernel que ejecutará en GPU ------------------------
  module = SourceModule("""
  __global__ void kernel_cinetica(int n, int *mgpu, float *vgpu, float * egpu)
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if( idx < n )
    {
      egpu[idx] = 0.5 * (mgpu[idx] * pow(vgpu[idx], 2));
    }
  }
  """) 

  kernel = module.get_function("kernel_cinetica")

  tiempo_gpu = datetime.now()

  # GPU ------------------------------------------------------------------------
  dim_hilo = 256
  dim_bloque = numpy.int( (cantObjetos + dim_hilo-1) / dim_hilo )

  kernel(numpy.int32(cantObjetos), masa_gpu, velocidad_gpu, energia_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )
  tiempo_gpu = datetime.now() - tiempo_gpu

  # Copio el resultado desde la memoria GPU ------------------------------------
  cuda.memcpy_dtoh(energia_cpu, energia_gpu)

  tiempo_total = datetime.now() - tiempo_total

  print( "Cantidad de elementos: ", cantObjetos )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )
  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  

except ValueError as valerr:
  print (valerr)
# except: 
#   print("Houston we have a problem!")


---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectoes A, B y R.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

Las conclusiones son explicadas en clase....


---
# 6 Bibliografia

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[6] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)


